**LONDON CRIME DATA**

This data counts the number of crimes at two different geographic levels of London (LSOA and borough) by year, according to crime type. Includes data from 2008 to present

**IMPORT BIGQUERY PACKAGE**

In [50]:
from google.cloud import bigquery

**MAKE A CLIENT OBJECT**

In [51]:
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


**CONNECT TO LONDON CRIME DATASET**

In [52]:
dataset_ref = client.dataset('london_crime', project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

**CHECK NUMBER OF TABLES IN THIS DATASET**

In [53]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

crime_by_lsoa


**CONNECT TO CRIME_BY_LSOA TABLE**

In [54]:
table_ref = dataset_ref.table('crime_by_lsoa')
table = client.get_table(table_ref)

In [55]:
table.schema

[SchemaField('lsoa_code', 'STRING', 'NULLABLE', 'Lower Layer Super Output Area code according to the Office for National Statistics', (), None),
 SchemaField('borough', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('major_category', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('minor_category', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('value', 'INTEGER', 'NULLABLE', 'Summary of the number of crimes for the month', (), None),
 SchemaField('year', 'INTEGER', 'NULLABLE', '', (), None),
 SchemaField('month', 'INTEGER', 'NULLABLE', '', (), None)]

**EXPLORE CONTENT IN THIS TABLE, LET'S SEE THE FIRST 10 ROWS**

In [56]:
client.list_rows(table, max_results=10).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000478,Brent,Sexual Offences,Rape,0,2008,1
1,E01000619,Brent,Sexual Offences,Rape,1,2008,1
2,E01000619,Brent,Sexual Offences,Rape,0,2008,7
3,E01000156,Barnet,Sexual Offences,Rape,0,2008,11
4,E01000156,Barnet,Sexual Offences,Rape,0,2008,7
5,E01000384,Bexley,Sexual Offences,Rape,1,2008,1
6,E01001301,Ealing,Sexual Offences,Rape,1,2008,1
7,E01001295,Ealing,Sexual Offences,Rape,0,2008,1
8,E01000291,Barnet,Sexual Offences,Rape,0,2008,8
9,E01000630,Brent,Sexual Offences,Rape,1,2008,2


**LET'S SEE HOW MANY BOROUGH THIS DATASET CONTAINED**

In [57]:
query = """
        SELECT borough
        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
        GROUP BY borough
        ORDER BY borough
        """
result = client.query(query).result().to_dataframe()
result.head(50)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough
0,Barking and Dagenham
1,Barnet
2,Bexley
3,Brent
4,Bromley
5,Camden
6,City of London
7,Croydon
8,Ealing
9,Enfield


**WHEN IS THIS DATA COLLECTED AND WHEN IS THE LAST DATA RECORDED**

In [58]:
query = """
        SELECT MIN(year) start_year, MAX(year) end_year
        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
        """
result = client.query(query).to_dataframe()
result.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,start_year,end_year
0,2008,2016


**WHICH BOROUGH HAS THE MOST NUMBER OF CRIMINAL CASES FROM 2008 TO 2016**

In [59]:
query = """
        SELECT borough, SUM(value) tot_crime
        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
        GROUP BY borough
        ORDER BY tot_crime DESC
        """
result = client.query(query).to_dataframe()
result.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough,tot_crime
0,Westminster,455028
1,Lambeth,292178
2,Southwark,278809
3,Camden,275147
4,Newham,262024
5,Croydon,260294
6,Ealing,251562
7,Islington,230286
8,Tower Hamlets,228613
9,Brent,227551


**WHAT MONTH IN A BOROUGH RECORDED A HIGHER NUMBER OF CRIMINAL CASES COMPARED TO OTHER TIMES**

In [60]:
query = """
        WITH time_crimes AS (
        SELECT borough, month, year, SUM(value) tot_num
        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
        GROUP BY borough, month, year
        )
        SELECT *
        FROM time_crimes
        WHERE tot_num = (
            SELECT MAX(tot_num)
            FROM time_crimes AS tc
            WHERE time_crimes.borough = tc.borough
        )
        ORDER BY tot_num DESC
        """
result = client.query(query).to_dataframe()
result.head(35)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough,month,year,tot_num
0,Westminster,7,2012,5428
1,Camden,3,2011,3196
2,Lambeth,7,2016,3161
3,Southwark,6,2008,3116
4,Ealing,7,2010,2996
5,Croydon,1,2012,2970
6,Newham,1,2008,2876
7,Tower Hamlets,7,2016,2783
8,Brent,10,2011,2625
9,Hackney,6,2016,2625


**FROM 2008 TO 2016 IN LONDON WHAT CRIMES PRINTED THE MOST NUMBER OF CASES**

In [61]:
query = """
        SELECT major_category, minor_category, SUM(value) num_crimes
        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
        GROUP BY major_category, minor_category
        ORDER BY num_crimes DESC
        """
result = client.query(query).to_dataframe()
result.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,major_category,minor_category,num_crimes
0,Theft and Handling,Other Theft,980085
1,Theft and Handling,Theft From Motor Vehicle,569956
2,Burglary,Burglary in a Dwelling,491282
3,Violence Against the Person,Harassment,458124
4,Violence Against the Person,Assault with Injury,451001
5,Drugs,Possession Of Drugs,431948
6,Violence Against the Person,Common Assault,413690
7,Theft and Handling,Theft From Shops,345142
8,Theft and Handling,Other Theft Person,308842
9,Criminal Damage,Criminal Damage To Motor Vehicle,265463


**WE WILL SORT THE CRIMES IN A BOROUGH BASED ON THE NUMBER OF CASES AND WILL DISPLAY THE THREE CRIMES WITH THE LARGEST NUMBER IN THAT AREA**

In [62]:
query = """
        SELECT borough, major_category, rank_per_borough, num_accident
        FROM(
            SELECT borough, major_category,
                RANK () OVER(PARTITION BY borough ORDER BY SUM(value) DESC) AS rank_per_borough,
                SUM(value) AS num_accident
            FROM `bigquery-public-data.london_crime.crime_by_lsoa`
            GROUP BY borough, major_category
        )
        WHERE rank_per_borough <= 3
        ORDER BY borough, rank_per_borough
        """
result = client.query(query).to_dataframe()
result.head(15)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough,major_category,rank_per_borough,num_accident
0,Barking and Dagenham,Theft and Handling,1,50999
1,Barking and Dagenham,Violence Against the Person,2,43091
2,Barking and Dagenham,Criminal Damage,3,18888
3,Barnet,Theft and Handling,1,87285
4,Barnet,Violence Against the Person,2,46565
5,Barnet,Burglary,3,36981
6,Bexley,Theft and Handling,1,40071
7,Bexley,Violence Against the Person,2,30037
8,Bexley,Criminal Damage,3,17244
9,Brent,Theft and Handling,1,72523


**DOES THE EVENT OF CRIME IN EACH BOROUGH HAVE INCREASED OR DECREASED EVERY YEAR?**

In [63]:
query = """
        SELECT borough,
            SUM(IF(year = 2008, value, 0)) tot_crimes_08,
            SUM(IF(year = 2009, value, 0)) tot_crimes_09,
            SUM(IF(year = 2010, value, 0)) tot_crimes_10,
            SUM(IF(year = 2011, value, 0)) tot_crimes_11,
            SUM(IF(year = 2012, value, 0)) tot_crimes_12,
            SUM(IF(year = 2013, value, 0)) tot_crimes_13,
            SUM(IF(year = 2014, value, 0)) tot_crimes_14,
            SUM(IF(year = 2015, value, 0)) tot_crimes_15,
            SUM(IF(year = 2016, value, 0)) tot_crimes_16
        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
        GROUP BY borough
        ORDER BY borough 
        """
result = client.query(query).to_dataframe()
result.head(35)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough,tot_crimes_08,tot_crimes_09,tot_crimes_10,tot_crimes_11,tot_crimes_12,tot_crimes_13,tot_crimes_14,tot_crimes_15,tot_crimes_16
0,Barking and Dagenham,17656,17713,17130,16686,15990,15759,15426,16346,16741
1,Barnet,23944,23449,23416,24007,23573,22510,22572,24036,24684
2,Bexley,15305,14445,13166,11325,11669,11552,11844,11990,12840
3,Brent,23504,24748,25512,27669,26357,23759,24426,24883,26693
4,Bromley,24015,21858,19572,19868,19622,18945,19668,20637,20164
5,Camden,32010,31612,31804,33956,31472,28435,27198,29477,29183
6,City of London,0,0,0,69,157,110,115,151,178
7,Croydon,29516,29784,29373,29830,30189,27444,27357,28089,28712
8,Ealing,28947,28617,31360,29537,28832,25917,24978,26247,27127
9,Enfield,21980,21147,20808,21041,21069,21574,21700,22076,22485


**WHAT CRIMES HAVE INCREASED AND DECREASED NUMBER OF CASE COMPARED IN 2008 WITH 2016 AND HOW MUCH CHANGES THE NUMBER OF CASES**

In [70]:
query = """
        SELECT major_category, minor_category, tot_crime_08, tot_crime_16,
        tot_crime_16 - tot_crime_08 AS change,
        ROUND(((tot_crime_16 - tot_crime_08)/tot_crime_16)*100,2) perc_change
        FROM(
            SELECT major_category, minor_category,
                SUM(IF(year = 2008, value, null)) AS tot_crime_08,
                SUM(IF(year = 2016, value, null)) AS tot_crime_16
            FROM `bigquery-public-data.london_crime.crime_by_lsoa`
            WHERE value > 0
            GROUP BY major_category, minor_category
        )
        ORDER BY perc_change DESC
        """
result = client.query(query).to_dataframe()
result.head(35)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,major_category,minor_category,tot_crime_08,tot_crime_16,change,perc_change
0,Theft and Handling,Motor Vehicle Interference & Tampering,3868,11438.0,7570.0,66.18
1,Violence Against the Person,Wounding/GBH,8701,23525.0,14824.0,63.01
2,Violence Against the Person,Harassment,43389,78676.0,35287.0,44.85
3,Violence Against the Person,Common Assault,37356,64440.0,27084.0,42.03
4,Other Notifiable Offences,Other Notifiable,9491,15205.0,5714.0,37.58
5,Theft and Handling,Theft From Shops,32857,46957.0,14100.0,30.03
6,Theft and Handling,Other Theft Person,24453,34868.0,10415.0,29.87
7,Violence Against the Person,Other violence,7721,10588.0,2867.0,27.08
8,Theft and Handling,Theft/Taking of Pedal Cycle,15895,18001.0,2106.0,11.70
9,Theft and Handling,Other Theft,99280,103807.0,4527.0,4.36
